In [162]:
import time
start_time = time.time()

import numpy as np
import pandas as pd
import math as mt

from nltk.stem.porter import *
stemmer = PorterStemmer()
import re

import random

random.seed(230)

# load csv data
df_train = pd.read_csv('./input/train_microset.csv', encoding="ISO-8859-1")
df_test = pd.read_csv('./input/test.csv', encoding="ISO-8859-1")
df_attributes = pd.read_csv('./input/attributes.csv')
num_train = df_train.shape[0]   # get the length of df_train
print(str(num_train) + " rows read from train.csv")


6 rows read from train.csv


In [163]:
# concat train and test data
df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
df_all = pd.merge(df_all, df_attributes, how='left', on='product_uid')
num_all = df_all.shape[0]
print(str(num_all) + " rows read from test.csv + train.csv")

3456425 rows read from test.csv + train.csv


In [164]:
# FOR TESTING
df_all = df_train
#Parameters
numberBands = 3 #9
shingleSize = 5
numberHashFunctions = 5#100

In [165]:
def str_stemmer(s):
    return " ".join([stemmer.stem(word) for word in s.lower().split()])

def str_common_word(str1, str2):  # this function not used in current example
    return sum(int(str2.find(word)>=0) for word in str1.split())


In [166]:
def shingles(s, k = shingleSize):
    return [s[i:i + k] for i in range(len(s) - k + 1)]

In [167]:
# apply str_stemmer to all search terms - converts to lower case
df_all['search_term'] = df_all['search_term'].map(lambda x:str_stemmer(x))

# obtain shingles to all search terms
df_all['shingles'] = df_all['search_term'].map(shingles)

In [168]:
df_all

,id,product_uid,product_title,search_term,relevance,shingles
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angl bracket,3.0,"[angl , ngl b, gl br, l bra, brac, brack, rac..."
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,ngl bracket,2.5,"[ngl b, gl br, l bra, brac, brack, racke, acket]"
2,4,100001,Simpson Strong-Tie 12-Gauge Angle,angl bracket,3.0,"[angl , ngl b, gl br, l bra, brac, brack, rac..."
3,36,100011,Toro Personal Pace Recycler 22 in. Variable Sp...,brigg and stratton lawn mower,3.0,"[brigg, rigg , igg a, gg an, g and, and , and..."
4,37,100011,Toro Personal Pace Recycler 22 in. Variable Sp...,hondag mower,3.0,"[honda, ondag, ndag , dag m, ag mo, g mow, mo..."
5,38,100011,Toro Personal Pace Recycler 22 in. Variable Sp...,honda mower,2.0,"[honda, onda , nda m, da mo, a mow, mowe, mower]"


In [169]:
# create characteristic matrix
def bitvector(i, shingles, charMatrix):
    for shingle in shingles:
        charMatrix.ix[shingle, i] = 1

In [170]:
charMatrix = pd.DataFrame(columns = df_all['id'])
df_all.apply(lambda x: bitvector(x['id'], x['shingles'], charMatrix), axis=1)
charMatrix.fillna(0, inplace= True)
countShingles = len(charMatrix)
countDocuments = len(charMatrix.columns)
charMatrix['index'] = range(0, countShingles)
charMatrix

id,2,3,4,36,37,38,index
angl,1,0,1,0,0,0,0
ngl b,1,1,1,0,0,0,1
gl br,1,1,1,0,0,0,2
l bra,1,1,1,0,0,0,3
brac,1,1,1,0,0,0,4
brack,1,1,1,0,0,0,5
racke,1,1,1,0,0,0,6
acket,1,1,1,0,0,0,7
brigg,0,0,0,1,0,0,8
rigg,0,0,0,1,0,0,9


In [171]:
#creating random hash functions, modular hashing
hashFunctions = pd.DataFrame(np.random.randint(low=1, high=countShingles, size=(2, numberHashFunctions)))
hashFunctions

,0,1,2,3,4
0,25,13,8,14,39
1,11,5,12,5,28


In [172]:
def calculateHash(row, hashRow):
    return (row['index'] * hashRow[0] + hashRow[1]) % countShingles

def createHash(row, minhashTable):
    hashColumn = str(row[0])+ "x + "+ str(row[1]) + " mod " + str(countShingles)
    minhashTable[hashColumn] = minhashTable.apply(calculateHash, args=(row, ), axis=1)

# create a hash column for each hash function generated
hashFunctions.apply(createHash, args=(charMatrix,))
charMatrix
    

id,2,3,4,36,37,38,index,25x + 11 mod 43,13x + 5 mod 43,8x + 12 mod 43,14x + 5 mod 43,39x + 28 mod 43
angl,1,0,1,0,0,0,0,11,5,12,5,28
ngl b,1,1,1,0,0,0,1,36,18,20,19,24
gl br,1,1,1,0,0,0,2,18,31,28,33,20
l bra,1,1,1,0,0,0,3,0,1,36,4,16
brac,1,1,1,0,0,0,4,25,14,1,18,12
brack,1,1,1,0,0,0,5,7,27,9,32,8
racke,1,1,1,0,0,0,6,32,40,17,3,4
acket,1,1,1,0,0,0,7,14,10,25,17,0
brigg,0,0,0,1,0,0,8,39,23,33,31,39
rigg,0,0,0,1,0,0,9,21,36,41,2,35


In [173]:
hashFunctionNames = charMatrix.columns.values[countDocuments+1:]
documentIndices = charMatrix.columns.values[0:countDocuments]

hashFunctions = charMatrix.ix[:,countDocuments+1:]
documents = charMatrix.ix[:,0:countDocuments]

#creation of signature matrix
sigMatrix = pd.DataFrame(2*countShingles,index=hashFunctionNames, columns=documentIndices)

def compareHashes(hashColumn, shingle, docMatch):
    hashedValue = hashFunctions.ix[shingle,hashColumn.name]
    signedValue = sigMatrix.ix[hashColumn.name,docMatch]
    return min(hashedValue, signedValue);

def calculateSignatureMat(docRow, sigMatrix):
    matches = docRow[docRow == 1].index.tolist()
    for match in matches:
        sigMatrix[match] = hashFunctions.apply(compareHashes, args=(docRow.name, match,))

documents.apply(calculateSignatureMat, args=(sigMatrix,), axis=1)

sigMatrix

,2,3,4,36,37,38
25x + 11 mod 43,0,0,0,2,1,4
13x + 5 mod 43,1,1,1,2,0,4
8x + 12 mod 43,1,1,1,0,2,2
14x + 5 mod 43,3,3,3,0,7,6
39x + 28 mod 43,0,0,0,2,5,1


In [174]:
#calculation of the height of the band
rowsPerBand = mt.ceil(numberHashFunctions / numberBands)

#set number of buckets for LSH
numberHashedBuckets = rowsPerBand*5;

#generation of random hashing functions for each band
lshFunctions = pd.DataFrame(np.random.randint(low=1, high=numberHashedBuckets, size=(numberBands, rowsPerBand)))
lshFunctions

,0,1
0,7,6
1,9,5
2,5,1


In [175]:
#set of pairs found to be similar by LSH
candidatePairs = set()

#buckets for each band
buckets = pd.DataFrame(index=np.arange(numberBands), columns=np.arange(numberHashedBuckets))

def calculateHashedBucket(subColumn,index):
    sum = 0
    for i in range(0, len(subColumn)):
        sum += lshFunctions[i][index]*subColumn[i]
    return sum % numberHashedBuckets

def lsh(sigColumn):
    i = 0
    while i < len(sigColumn):
        hashFunctionIndex = i // rowsPerBand
        hashedBucket = calculateHashedBucket(sigColumn[i:i+rowsPerBand], hashFunctionIndex)
        if isinstance(buckets.ix[hashFunctionIndex,hashedBucket], list):
            for doc in buckets.ix[hashFunctionIndex,hashedBucket]:
                candidatePairs.add((doc, sigColumn.name))
            buckets.ix[hashFunctionIndex,hashedBucket].append(sigColumn.name)
        elif np.isnan(buckets.ix[hashFunctionIndex,hashedBucket]):
            buckets.ix[hashFunctionIndex,hashedBucket] = [sigColumn.name]
        i += rowsPerBand

#calculation of LSH
sigMatrix.apply(lsh)
buckets

0
0
--------------
2
1
--------------
4
2
--------------
0
0
--------------
2
1
--------------
4
2
--------------
0
0
--------------
2
1
--------------
4
2
--------------
0
0
--------------
2
1
--------------
4
2
--------------
0
0
--------------
2
1
--------------
4
2
--------------
0
0
--------------
2
1
--------------
4
2
--------------


,0,1,2,3,4,5,6,7,8,9
0,NaN,NaN,[38],NaN,NaN,NaN,"[2, 3, 4, 36]",[37],NaN,NaN
1,[36],NaN,NaN,[37],"[2, 3, 4]",NaN,NaN,NaN,[38],NaN
2,"[2, 3, 4, 36]",NaN,NaN,NaN,NaN,"[37, 38]",NaN,NaN,NaN,NaN


In [176]:
candidatePairs

{(2, 3), (2, 4), (2, 36), (3, 4), (3, 36), (4, 36), (37, 38)}

In [177]:
def jaccardSim(doc1,doc2):
    match = 0
    for i in range(0, len(doc1)):
        if doc1[i] == doc2[i]:
            match+=1
    return match / len(doc1)

pairsToRemove = set()
for (docA, docB) in candidatePairs:
    similarity = jaccardSim(sigMatrix[docA], sigMatrix[docB])
    if similarity < 0.8:
        pairsToRemove.add((docA, docB))

finalPairs = candidatePairs.difference(pairsToRemove)
    

In [178]:
len(pairsToRemove)

4

In [179]:
finalPairs

{(2, 3), (2, 4), (3, 4)}